In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import gmean
from sklearn.cluster import KMeans, k_means
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Import the data and create a pandas dataframe. This uses a *.csv file and the example below is for a file located locally on the user's computer. 

df = pd.read_csv(r'C:\Users\...\[Filename].csv')

In [ ]:
df.head() #To visualize the first 5 rows of the dataframe

In [ ]:
#Remove the columns that are not used in the PCA and kmeans analysis

df2=df[df.columns[~df.columns.isin(['Sample','Latitude', 'Longitude', 'Easting', 'Northing'])]]
df2

In [ ]:
#Create a numpy array from the new dataframe

arr=df2.to_numpy()
arr

In [ ]:
#Get the number of elements in the array

arr.shape

# Centred-log ratio transformation

In [ ]:
#Get the geometric mean of each row

arr_2=gmean(arr, axis=1)
arr_2

In [ ]:
arr_2.shape

In [ ]:
arrT=arr.T               #Transpose matrix
arrT

In [ ]:
arrT.shape

In [ ]:
#Apply the clr transform

arr_clr=np.log(arrT/arr_2)
arr_clr

In [ ]:
arr_clrT=arr_clr.T   #Transpose again to get back to the original array shape
arr_clrT

In [ ]:
arr_clrT.shape #Verify shape

# PCA Analysis and kmeans clustering

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
from sklearn.preprocessing import StandardScaler

X= arr_clrT

# Preprocessing using scikit-learn tools
Xs=StandardScaler().fit_transform(X) #Standardize the data to zero mean and unit variance

print(Xs)

In [ ]:
Xs.mean()

In [ ]:
Xs.std()

In [ ]:
#Export the results of the standardization

SD_data = pd.DataFrame(Xs)
print(SD_data)
SD_data.to_csv(r'C:\Users\iamma\Documents\SD_data.csv')

In [ ]:
# Flatten the 2D array into a 1D array
Xs_flat = Xs.flatten()

# Create histogram with optimal bin size to visualize the standardized data
n, bins, patches = plt.hist(Xs_flat, bins='auto', density=True, alpha=0.9, edgecolor='black', linewidth=1.2)

# Add labels and title to plot
plt.xlabel('z values')
plt.ylabel('Frequency')

plt.show()

In [ ]:
print(bin_size)

In [ ]:
import scipy.stats as stats

# Create a probability plot without a title and avoid the AttributeError with ppf
fig, ax = plt.subplots()
stats.probplot(Xs_flat, plot=ax)
ax.set_title('')  # remove the default title
plt.show()

In [ ]:
n_components = X.shape[1]

#Running PCA on all components
pca=PCA(n_components=n_components, svd_solver='randomized')
X_r=pca.fit(Xs).transform(Xs)

#calculating the 95% variance
total_variance = sum(pca.explained_variance_)
print('Total variance in the dataset is:', total_variance)
var_95 = total_variance*0.95
print('The 95% variance is: ', var_95)
print('')

#Creating a df with the components and explained variance
a = zip(range(0,n_components), pca.explained_variance_)
a = pd.DataFrame(a, columns=['PCA Comp', 'Explained Variance'])

#Trying to find 95%... 
print('Variance explained with 2 components:', sum(a['Explained Variance'][0:2]))
print('Variance explained with 3 components:', sum(a['Explained Variance'][0:3]))
print('Variance explained with 4 components:', sum(a['Explained Variance'][0:4]))
print('Variance explained with 5 components:', sum(a['Explained Variance'][0:5]))


In [ ]:
print(a)

In [ ]:
print(pca.explained_variance_ratio_)

In [ ]:
# calculate the cumulative sum of explained variance ratio
cumulative_var_ratio = np.cumsum(pca.explained_variance_ratio_)

# find the index where the cumulative sum first exceeds 0.95
threshold_idx = np.where(cumulative_var_ratio >= 0.95)[0]

if threshold_idx.size == 0:
    # if no index is found, set the threshold index to the last index
    threshold_idx = len(cumulative_var_ratio) - 1
else:
    # get the first index where the cumulative sum exceeds 0.95
    threshold_idx = threshold_idx[0]

#Make a scree plot
fig, (ax1)=plt.subplots(1, figsize=(16,6))
Xaxis=np.arange(len(a))
ax1.plot(Xaxis,pca.explained_variance_ratio_, linewidth=2, c='r')
ax1.set_xticks(Xaxis)
ax1.set_xticklabels(Xaxis+1)
plt.xlabel('n components')
plt.ylabel('explained ratio')

# add a horizontal dotted line at 95% explained variance ratio
ax1.axhline(y=pca.explained_variance_ratio_[threshold_idx], linestyle=':', label='95% explained variance', c='blue')
plt.legend(prop=dict(size=12))
plt.show()

In [ ]:
# Running PCA again, this time with 4 components

pca=PCA(n_components=4, svd_solver='randomized')
X_r2=pca.fit(Xs).transform(Xs)

inertia=[]

#Determining the nb# of kmeans clusters

no_of_clusters=range(2,20) #[2,3,4,5...]
Index=[]    #Creates an empty list

for f in no_of_clusters:
    kmeans = KMeans(n_clusters=f)
    kmeans = kmeans.fit(X_r2)
    u = kmeans.inertia_
    inertia.append(u)
    print('The inertia for :', f, 'clusters is', u)
    


In [ ]:
#Making a scree plot of inertia scores; the elbow method to detemrine the number of clusters. 
#In this case, there is a slight break in slope at 4 and 6 clusters

fig, (ax1)=plt.subplots(1, figsize=(16,6))
xx=np.arange(len(no_of_clusters))
ax1.plot(xx, inertia)
ax1.set_xticks(xx)
ax1.set_xticklabels(no_of_clusters)
plt.xlabel('Nb of clusters')
plt.ylabel('Inertia score')
plt.axvline(2, linestyle=':', color='green')   #plot a line for Nb. of clusters based on the 'elbow method'
plt.axvline(4, linestyle=':', color='orange')
plt.savefig('Screeplot.png', format='png')

In [ ]:
#Another method to deterine the number of clusters (the Calinski-Harabasz score)

no_of_clusters=range(2,20) #[2,3,4,5...]
Index2=[]    #Creates an empty list

for i in no_of_clusters:
    kmeans=KMeans(n_clusters=i, random_state=2)
    kmeans=kmeans.fit(X_r2)
    CH=metrics.calinski_harabasz_score(X_r2, kmeans.labels_)
    Index2.append(CH)   #This will populate the inertia list with u
    print("The Calinski Harabasz score for :", i, "clusters is:", CH)

In [ ]:
#Running K means on 6 clusters based on the two methods above as well as on local geological knowledge

kmeans2=KMeans(n_clusters=6)
kmeans2=kmeans2.fit(X_r2)


clusters2=kmeans2.predict(X_r2)

list_clusters=['Cluster 1','Cluster 2', 'Cluster 3', 'Cluster 4', 'Cluster 5', 'Cluster 6']

#calculating the counts of the clusters
unique, counts=np.unique(kmeans2.labels_, return_counts=True)
counts=counts.reshape(1,6)   #one row and six columns

#Creating a dataframe
countscldf2=pd.DataFrame(counts, columns=list_clusters)

#display
countscldf2

In [ ]:
loadings = pd.DataFrame(pca.components_.T, columns=['PC1', 'PC2', 'PC3', 'PC4'], index=['SiO2', 'Al2O3', 'Fe2O3','MgO','CaO','Na2O', 'K2O', 'TiO2', 'P2O5', 'MnO', 'Cr2O3'])
loadings.index.name='Elements' #To name the index column
loadings['Elements']=loadings.index    #the index is copied on to a new column with column name
loadings = loadings.reset_index(drop=True) #the index replaced with sequence of numbers
loadings

In [ ]:
PC1_sorted=loadings.sort_values(by=['PC1'])
PC1_sorted

In [ ]:
plt.bar((PC1_sorted['Elements']), PC1_sorted['PC1'], linewidth=2., color='grey')
plt.ylim((-0.7, 0.7)) 
plt.ylabel('PC 1 loadings', fontsize=15)
plt.axhline(0,linestyle='-', c='black')
plt.savefig('PC_loadings.png', format='png')
plt.show()

In [ ]:
PC2_sorted=loadings.sort_values(by=['PC2'])
PC2_sorted

In [ ]:
plt.bar((PC2_sorted['Elements']), PC2_sorted['PC2'], linewidth=2., color='grey')
plt.ylim((-0.8, 0.8)) 
plt.ylabel('PC 2 loadings', fontsize=15)
plt.axhline(0,linestyle='-', c='black')
#plt.savefig('PC_loadings.png', format='png')
plt.show()

In [ ]:
a_=pca.components_.copy()
a_T=a_.T
print(a_T)

In [ ]:
y_num = clusters2

target_names = list_clusters

PC1_Explained = pca.explained_variance_ratio_[0].round(4)*100
PC2_Explained = pca.explained_variance_ratio_[1].round(4)*100
PC3_Explained = pca.explained_variance_ratio_[2].round(4)*100

#PLotting the data (pc1 V. pc2)
plt.figure()
plt.figure(figsize=(12,8))
colors = ['black','blue', 'green', 'orange', 'red', 'brown']
markers=['o','^','x', '.', 's', 'p']
lw=2

# Get the absolute maximum value of PC1 and PC2 scores
PC1_abs_max = np.abs(X_r[:, 0]).max()
PC2_abs_max = np.abs(X_r[:,1]).max()
n = pca.components_.T.shape[0]

#Assign both color and symbol to each cluster
for color, i, marker, m, target_name in zip(colors, [0,1,2,3,4,5], markers, [0,1,2,3,4,5], target_names):
    plt.scatter(X_r2[y_num == i, 0], X_r2[y_num == i, 1], color=color, marker=marker, alpha=.8, lw=lw, label=target_name)

#Add the loading vectors scaled to the PC scores
for l in range(n):
    plt.quiver(0, 0, pca.components_.T[l,0]*PC1_abs_max, pca.components_.T[l,1]*PC2_abs_max,color = 'b',alpha = 0.5, width=0.002, headwidth=4, headlength=4, angles='xy', scale_units='xy', scale=1)

plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.xlabel(f'PC 1 [{PC1_Explained:.2f}%]', fontsize=14) #using the .2f format specifier within the f-strings formats the floating-point numbers with two decimal places
plt.ylabel(f'PC 2 [{PC2_Explained:.2f}%]', fontsize=14)
plt.axvline(0,linestyle=':', c='black')
plt.axhline(0, linestyle=':', c='black')


plt.text(a_T[0,0]*PC1_abs_max, a_T[0,1]*PC2_abs_max, 'Si$O_2$')
plt.text(a_T[1,0]*PC1_abs_max, a_T[1,1]*PC2_abs_max, '$Al_2$$O_3$')
plt.text(a_T[2,0]*PC1_abs_max, a_T[2,1]*PC2_abs_max, '$Fe_2$$O_3$')
plt.text(a_T[3,0]*PC1_abs_max, a_T[3,1]*PC2_abs_max, 'MgO')
plt.text(a_T[4,0]*PC1_abs_max, a_T[4,1]*PC2_abs_max-0.2, 'CaO')
plt.text(a_T[5,0]*PC1_abs_max, a_T[5,1]*PC2_abs_max, '$Na_2$O')
plt.text(a_T[6,0]*PC1_abs_max, a_T[6,1]*PC2_abs_max, '$K_2$O')
plt.text(a_T[7,0]*PC1_abs_max, a_T[7,1]*PC2_abs_max, 'Ti$O_2$')
plt.text(a_T[8,0]*PC1_abs_max-0.5, a_T[8,1]*PC2_abs_max, '$P_2$$O_5$')
plt.text(a_T[9,0]*PC1_abs_max, a_T[9,1]*PC2_abs_max, 'MnO')
plt.text(a_T[10,0]*PC1_abs_max, a_T[10,1]*PC2_abs_max, '$Cr_2$$O_3$')

plt.savefig('PC1_2_.png', format='png')
plt.show()


In [ ]:
#PLotting the data (pc1 v pc3)
plt.figure()
plt.figure(figsize=(12,8))
colors = ['black','blue', 'green', 'orange', 'red', 'brown']
markers=['o','^','x', '.', 's', 'p']
lw=2

# Get the absolute maximum value of PC1 and PC2 scores
PC1_abs_max = np.abs(X_r[:, 0]).max()
PC3_abs_max = np.abs(X_r[:,2]).max()
n = pca.components_.T.shape[0]

#Assign both color and symbol to each cluster
for color, i, marker, m, target_name in zip(colors, [0,1,2,3,4,5], markers, [0,1,2,3,4,5], target_names):
    plt.scatter(X_r2[y_num == i, 0], X_r2[y_num == i, 2], color=color, marker=marker, alpha=.8, lw=lw, label=target_name)

#Add the loading vectors scaled to the PC scores
for l in range(n):
    plt.quiver(0, 0, pca.components_.T[l,0]*PC1_abs_max, pca.components_.T[l,2]*PC3_abs_max,color = 'b',alpha = 0.5, width=0.002, headwidth=4, headlength=4, angles='xy', scale_units='xy', scale=1)

plt.legend(loc='best', shadow=False, scatterpoints=1)
#plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.6)  #This would used to put the legend outside the plot
plt.xlabel(f'PC 1 [{PC1_Explained}%]', fontsize=14)
plt.ylabel(f'PC 3 [{PC3_Explained}%]', fontsize=14)
plt.axvline(0,linestyle=':', c='black')
plt.axhline(0, linestyle=':', c='black')


plt.text(a_T[0,0]*PC1_abs_max, a_T[0,2]*PC3_abs_max, 'Si$O_2$')
plt.text(a_T[1,0]*PC1_abs_max, a_T[1,2]*PC3_abs_max, '$Al_2$$O_3$')
plt.text(a_T[2,0]*PC1_abs_max, a_T[2,2]*PC3_abs_max, '$Fe_2$$O_3$')
plt.text(a_T[3,0]*PC1_abs_max, a_T[3,2]*PC3_abs_max, 'MgO')
plt.text(a_T[4,0]*PC1_abs_max, a_T[4,2]*PC3_abs_max, 'CaO')
plt.text(a_T[5,0]*PC1_abs_max, a_T[5,2]*PC3_abs_max, '$Na_2$O')
plt.text(a_T[6,0]*PC1_abs_max, a_T[6,2]*PC3_abs_max, '$K_2$O')
plt.text(a_T[7,0]*PC1_abs_max, a_T[7,2]*PC3_abs_max, 'Ti$O_2$')
plt.text(a_T[8,0]*PC1_abs_max, a_T[8,2]*PC3_abs_max, '$P_2$$O_5$')
plt.text(a_T[9,0]*PC1_abs_max, a_T[9,2]*PC3_abs_max, 'MnO')
plt.text(a_T[10,0]*PC1_abs_max, a_T[10,2]*PC3_abs_max, '$Cr_2$$O_3$')

#plt.savefig('PC1_2_.pdf')
#plt.savefig('PC1_2_.png', format='png')
plt.show()



In [ ]:
#Create a new dataframe of the PC scores

df_PCscores=pd.DataFrame(X_r2, columns=['PC1', 'PC2', 'PC3', 'PC4'])
df_PCscores

In [ ]:
#Insert the kmeans labels into the new dataframe

df_PCscores.insert(4, "k_means_label", kmeans2.labels_)
#df_PCscores.insert(4, "Sample", df)
df_PCscores.head(30)

In [ ]:
#Insert the sample numbers into the new dataframe

extracted_col = df["Sample"]
  
df_PCscores.insert(5, "Sample", extracted_col)

df_PCscores.head(30)

In [ ]:
#Export the dataframe containing all results as a *.csv file for use externally such as to plot results in a GIS (for this, add the coordinates too)

df_PCscores.to_csv('df_Rice_etal_Major_Oxides_PC_kmeans_.csv', index=False)